In [1]:
from notebook.services.config import ConfigManager
cm = ConfigManager()


In [2]:
cm.update( 'rise', {"theme": "sky"} )

{'theme': 'sky'}

# <center> Limit the Guesswork in Fantasy Baseball by Using Python for Data Analysis </center>

Nicholas Marey<br>
July 11, 2019

## Topics

* About me
* What is baseball?
* What is daily fantasy sports
* Data
* Naive model (rolling average)
* Univariate model (simple exponential smoothing)
* Non-parametric method (kernel density estimator)
* Bayesian analysis (pyStan)
* Prediction model (general additive model)
* Optimization
* Analysis / Data delivery (flask web app)
* Next steps

## About Me

* Former chemical engineer
* Worked as a process engineer at a factory that made toilet paper
* Masters of applied statistics
* Data scientist?

## What is Baseball???

<center> It is a child's game played by grown men who sometimes act like children </center>
<center> <img src='yolmer.png'> </center>

<center>Its so beautiful that it is basically art.</center>
<table><tr><td><img src='yasiel_puig_fight_small.png'></td><td><img src='puig_art.jpg'></td></tr></table>


### <center>Baseball is a game of failure</center>

If a player can get a hit __30%__ of the time for their entire career will they would be on par with players in the __Hall of Fame__.

Getting on base an average of __372__ times per __1,000__ attempts is a __Hall of Fame__ level output. 




### <center> Baseball is very quirky </center>
* Other than the basepath there is no standard field side
* There is no time clock
* The offense does not have possession of the baseball. The pitcher initiates and dictates the play and the defense has possession of the ball.  
* For better or worse, the manager wears a uniform, beer belly and all.
* They play almost every day.

<center> Baseball is a great sport to apply statistical analysis due to its series of one on one interactions, in a finite number of states </center>

## Fantasy and Daily Fantasy Sports, A Primer

### <center> Fantasy sports </center>

A game where individuals assemble imaginary teams of real players of a professional sport.






### <center > Daily Fantasy Sports </center>

Daily fantasy sports (DFS) consist of short, typically one day contests, where participants have an assemble a team of players, who have an associate cost or \"salary\" and earn points based upon actual statistical performance of the real players.

The participant must put together a team while staying under a salary cap.




## Timeline

**1980** First fantasy baseball league is created at a hotel in Manhattan.

**October 13, 2006** The Unlawful Internet Gambling Enforcement Act (UIGEA) was signed into law. While the bill crippled the online poker industry, it contained explicit language that legalized daily fantasy sports.

**January 2009** FanDuel was founded

**January 2012** DraftKings was launched



The first fantasy baseball league was created in a manhattan hotel in 1980. From there it rapidly gained popularity

## Data, What's Up With it, And How Does I Get Sum???

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
from calendar import month_abbr

def br_scrapper(player, pos, year):
    url = "https://www.baseball-reference.com/players/gl.fcgi?id={}&t={}&year={}".format(player, pos, year)
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page,'html.parser')
    
    header = soup.find('table', id="batting_gamelogs").find('thead').find('tr').text.split('\n')
    #header = soup.find('table', id="batting_gamelogs").find('tr').text.split('\n')
    header = header[2:-1] # drop first and last item
    header = ['team_homeORaway' if x=='' else x for x in header]
    
    table = soup.find('table', id="batting_gamelogs")
    table_rows = table.find('tbody').find_all('tr', id=re.compile("batting_gamelogs.\d{4}"))
    #table_rows
    
    data = []
    for row in table_rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append(cols)
    
    player_name = soup.find('div', itemtype='https://schema.org/Person').find('h1').text
    bats = soup.find('div', itemtype = 'https://schema.org/Person').text.split('\n')[8].split(':')[1].strip()
    
    data = pd.DataFrame(data=data, columns = header)
    
    data['Yr'] = year
    data['Player'] = player_name
    data['Bats'] = bats
    data['Month'] = data['Date'].str.split(" ").str[0]
    data['Day'] = data['Date'].str.split(" ").str[1]
    
    month_dict = dict((v,k) for k,v in enumerate(month_abbr))
    
    mth = []
    for i in data['Month']:
        mth.append(month_dict[i])
    
    data['Month'] = mth
    data['Day'] = data['Day'].astype(str).astype(int)
    
    data['dt'] = pd.to_datetime(data['Yr']*10000+data['Month']*100+data['Day'], format = '%Y%m%d')
    
    # data = data.drop(["Yr", "Month", "Day"], axis=1)
    
    data = data.drop(['DFS(DK)', 'DFS(FD)'], axis = 1)
    
    data[['AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB','IBB', 
          'SO', 'HBP', 'SH', 'SF', 'ROE', 'GDP', 'SB', 'SB', 'CS','BOP']] = data[['AB', 'R', 'H', '2B', '3B',
                                                                               'HR', 'RBI', 'BB', 'IBB',
                                                                               'SO', 'HBP', 'SH', 'SF', 'ROE',
                                                                               'GDP', 'SB', 'SB', 'CS', 'BOP']].astype(int)
    data[['BA', 'OBP', 'SLG', 'OPS','aLI', 'WPA', 'RE24']] = data[['BA', 'OBP', 'SLG', 'OPS','aLI', 'WPA', 'RE24']].astype(float)

    
    if pos == 'b':
        data['DFS(DK)'] = (data['H'] - data['2B'] - data['3B'] - data['HR'])*3 + data['2B']*5 + data['3B']*8 + data['HR']*10 + data['RBI']*2 + data['R']*2 + data['BB']*2 + data['HBP']*2 + data['SB']*5
        data['DFS(FD)'] = (data['H'] - data['2B'] - data['3B'] - data['HR'])*3 + data['2B']*6 + data['3B']*9 + data['HR']*12 + data['RBI']*3.5 + data['R']*3.2 + data['BB']*3 + data['HBP']*3 + data['SB']*6
    
    return(data)
    

# Rolling Average

What is it?

## Example

In [ ]:
test code

# Exponential Smoothing

What is it?

## Example

# Kernel Density Estimator

What is it?

## Example: What is the distribution of the data?

## Bayesian Analysis with pyStan
### Beta - Binomial Conjugate Pair

What is it?

## Example: predicting the probability a player will get a hit

# General Additive Model

What is it?

Example

# Putting it all in a flask web app

Process Flow

Get the data -> Store it in NoSQL -> Analyze with Python -> output in Flask and bokeh

<img src='https://bokeh.pydata.org/en/latest/_images/bokeh_serve.svg'>
Figure from Bokeh documentation

Example

# <center> The End </center>

Linkedin <br>
Github

# <center> References </center>

1. whatever
2. test
* test